# Improved_Diffusion的PyTorch代码逐行深入讲解

来自b站up主deep_thoughts 合集【PyTorch源码教程与前沿人工智能算法复现讲解】

P_58_Improved_Diffusion的PyTorch代码逐行深入讲解：

https://www.bilibili.com/video/BV1sG411s7vV/?spm_id_from=pageDriver&vd_source=18e91d849da09d846f771c89a366ed40

***论文***

Denoising Diffusion Probabilistic Models：

https://arxiv.org/pdf/2006.11239.pdf

Autoregressive Denoising Diffusion Models for Multivariate Probabilistic Time Series Forecasting：

http://proceedings.mlr.press/v139/rasul21a/rasul21a.pdf

Improved Denoising Diffusion Probabilistic Models：

https://arxiv.org/pdf/2102.09672.pdf

***代码***

improved-diffusion：

https://github.com/openai/improved-diffusion

# DDPM

## diffusion
### forward process（无参）
* beta schedule
* 基于x[0]和t采样出t时刻加噪后的样本x[t]
* 基于x[0]、x[t]、t计算出后验分布并采样得到去噪后的样本x[t-1]
* x[T]服从标准分布
### reverse process（含参）
* training
  * 将x[t]和t带入NN模型，来预测出x[t-1]的分布
    * 预测方差
    * 预测方差线性加权的权重
    * 固定的方差
      * betas
      * betas_bar
    * 预测噪声
    * 预测期望均值
    * 预测x[0]
  * MSE loss
  * 计算预测的分布与真实的后验分布之间的KL loss
  * 基于训练集和t的采样训练
    * 均匀采样t
    * 基于loss来进行重要性采样t
* infer
  * 自x[T]开始带入NN模型采样出x[T-1]...x[0]

## neural model
### attention-based UNet
* input blocks
  * ResBlock
    * x
    * emb
      * timestep
      * condition
  * MHSA Block
  * Downsample Block
* mid blocks
  * ResBlock
  * MHSA Block
  * ResBlock
* output blocks
  * ResBlock（与input blocks对应层的输出进行拼接作为输入）
  * MHSA block
  * Upsample Block
* output layer

# IDDPM 代码解读
https://github.com/openai/improved-diffusion

## 命令行巧妙传参及DataLoader

def create_argparser()

    """ 从字典中自动生成命令行传参的argument parser"""

    defaults = dict()  # 将参数定义为字典

    add_dict_to_argparser  # 函数巧妙传参

def create_named_schedule_sampler()

    """ 平均采样 或 基于二阶动量平滑loss采样 """
    
data = load_data()  # 加载数据

## Noise Scheduling

def create_model_and_diffusion()  # 定义模型和扩散过程

def create_gaussian_diffusion()  # 生成一个扩散过程的框架

def get_named_beta_schedule()  # 定义线性加噪方案 和 余弦加噪方案

def betas_for_alpha_bar()  # 根据 alpha 计算 beta

## Gussian Diffusion前向计算

class SpacedDiffusion(GaussianDiffusion)  # 在基础 diffusion过程 上进行优化

class GaussionDiffusion  # 最原始的 diffusion 框架代码

## q分布均值方差及其采样函数

def q_mean_variance()  # 基于 x0 和 t 算 x[t] 分布的均值和方差

def q_sample()  # 计算 q 即 x[t] 的采样值 / 重参数采样

def q_posterior_mean_variance  # 后验分布的均值和方差

## p分布均值方差及其推理迭代采样

def p_mean_variance() # 得到逆扩散过程的均值和方差

def _predict_xstart_from_eps() # 根据 x[t] t eps 预测 x0

def _predict_xstart_from_xprev() # 根据 x[t-1] 预测 x0

def _predict_eps_from_xstart() # 根据 x0 预测 eps

def p_sample() # 基于 x[t] 采样出 x[t-1]

## KL散度计算以及MSE loss

def _vb_terms_bpd()  # vb 变分下界 bpd (bit per dim)每个维度的比特数 
    
    # 真实的x[0] x[t] 和 t 去计算出 x[t-1] 的均值和方差
    
    # x[t] t 和预测的 x[0] 去计算出 x[t-1] 的均值和方差
    
    # p_theta 与 q 分布之间的 KL散度 对应着L[t-1]损失函数
    
    discretized_gaussian_log_likelihood  # 计算离散数似然 对应着 L[0] 损失函数
    
    # t=0时刻，用离散的高斯分布去计算似然
    
    # t>0时刻，直接用KL散度
    
def training_losses()  # 三种方案：只学习vb loss；只学习MSE loss；同时学习MSE和vb loss

## Attention-based U-Net 代码

nn.py

## 训练模型

In [ ]:
MODEL_FLAGS="--image_size 64 --num_channels 128 --num_res_blocks 3 --class_cond True"
DIFFUSION_FLAGS="--diffusion_steps 4000 --learn_sigma True --noise_schedule cosine --use_kl True"
TRAIN_FLAGS="--lr 1e-4 --batch_size 24"

python scripts/image_train.py --data_dir datasets/cifar_train $MODEL_FLAGS $DIFFUSION_FLAGS $TRAIN_FLAGS